In [ ]:
!pip install python-dotenv
!pip install jiwer
!pip install gradio
!pip install hopsworks

In [ ]:
# imports
import pandas as pd
import joblib
import numpy as np
import xgboost as xgb # XGboost
from sklearn import metrics
from sklearn.model_selection import  train_test_split,cross_val_score,KFold,GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,explained_variance_score,mean_squared_error
import warnings
import sys
import os
from time import time
warnings.filterwarnings("ignore")
import seaborn as sns
from matplotlib import pyplot
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
sys.path.append('/content/drive/MyDrive/ID2223-project/air quality prediction2')
from functions import *

Mounted at /content/drive


In [ ]:
import hopsworks
project = hopsworks.login() 
fs = project.get_feature_store() 

In [ ]:
feature_view = fs.get_feature_view(
    name = 'air_quality_fv',
    version = 1
)

In [ ]:
X_train, X_test, Y_train, Y_test = feature_view.train_test_split(0.2)
X_train = X_train.drop(columns=["date"]).fillna(0)
X_test = X_test.drop(columns=["date"]).fillna(0)
model = xgb.XGBRegressor()      

In [ ]:
X_test

,tempmin,feelslikemin,feelslike,dew,humidity,precipcover,windgust,sealevelpressure,visibility
2,-2.318942,-2.160226,-1.463164,-2.203428,-1.907011,-0.697194,-0.107090,1.601814,0.468961
7,-2.447172,-2.687673,-2.805366,-2.749503,-1.614535,-0.697194,1.315441,2.362294,0.573933
9,-1.918225,-1.876216,-1.969434,-3.394865,-3.104122,-0.697194,-0.575377,1.912214,0.836364
14,-1.982340,-1.835643,-1.333654,-1.582887,-1.111204,-0.697194,-1.441265,0.561974,-2.365294
19,-1.437365,-1.267623,-1.003990,-1.297439,-1.254041,-0.697194,-1.308731,0.903414,-0.843194
...,...,...,...,...,...,...,...,...,...
334,-1.789996,-1.916789,-1.781054,-1.371904,0.201537,-0.697194,0.635100,1.679414,0.836364
335,-2.014397,-2.160226,-1.922339,-1.396725,0.235546,-0.697194,0.564416,1.477654,0.836364
337,-1.164878,-1.037711,-1.227690,-0.714131,0.548428,-0.697194,-0.389829,1.105174,0.783878
355,-2.014397,-1.903265,-1.416070,-1.868336,-1.444491,-0.697194,0.219827,0.701654,0.731392


In [ ]:
#This part is used to get the parameter of XGB
learning_rate = [0.002,0.004,0.006,0.008,0.01,0.03,0.05]
n_estimators = [200,400,600,800,1000,1200,1400,1600,1800,2000]
max_depth = [3,6,10,15,20]
param_grid = dict(learning_rate = learning_rate,n_estimators = n_estimators,max_depth=max_depth )#Convert to dictionary format
 
kflod=KFold(n_splits=5,shuffle=True)

print("Begin Train")
grid_search = GridSearchCV(model,param_grid,scoring='neg_mean_squared_error',n_jobs=-1,cv=kflod)


grid_result = grid_search.fit(X_train, Y_train) 
print("Best: %f using %s" % (grid_result.best_score_,grid_search.best_params_))

means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean,param in zip(means,params):
    print("%f  with:   %r" % (mean,param))





In [ ]:
model = xgb.XGBRegressor(max_depth=6, learning_rate=0.09, n_estimators=1300,min_child_weight=5,silent=False, objective='reg:squarederror')
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
Y_pred=Y_pred.round().tolist()
Y_pred

In [ ]:
mae=mean_squared_error(Y_test, Y_pred)
mae

241.36486486486487

In [ ]:
metrics = explained_variance_score(Y_test, Y_pred, sample_weight=None, multioutput='uniform_average')
metrics

0.7275681585563579

In [ ]:
mr = project.get_model_registry()
model_dir="AIR_Forecast_Model"
if os.path.isdir(model_dir) == False:
   os.mkdir(model_dir)

joblib.dump(model, model_dir + "/AIR_Forecast_Model.pkl")   

input_schema = Schema(X_train)
output_schema = Schema(Y_train)
model_schema = ModelSchema(input_schema, output_schema)

AIR_Forecast_model = mr.sklearn.create_model(
    name="AIR_Forecast_Model", 
    metrics={"explained_variance_score" : metrics},
    model_schema=model_schema,
    description="AIR Forecast"
)

AIR_Forecast_model.save(model_dir)